In [1]:
import sklearn
import math
import pandas
import numpy
from sklearn.linear_model import LinearRegression

In [2]:
df = pandas.DataFrame({"x": numpy.random.rand(100)})
df["y"] = -df["x"] * df["x"] + 4
df["z"] = df["y"] * 2 + df["x"]

In [3]:
from skopt.space import Real, Integer
from skopt.utils import use_named_args

space = [
    Integer(0, 1, name="intercept"),
    Real(0, 1, "uniform", name="dummy"),
    Real(1, 5, "uniform", name="scale"),    
]

In [4]:
from skopt import gp_minimize
from skopt import forest_minimize

class MLModel(object):
    def __init__(self, df, params={}):
        self.params = params
        self.df = df
    
    def set_params(self, **params):
        print(params)
        self.params = params

    def set_dataftame(self, df):
        self.df = df
        
    def opt_fit(self, space):
        #gp = gp_minimize(self.objective, space, n_calls=50, random_state=0)
        
        @use_named_args(space)
        def my_objective(**params):
            self.set_params(**params)
            obj = self._train()
            return -obj
            
        gp = forest_minimize(func=my_objective, dimensions=space)
        return gp
       
    def objective(self, **params):
        self.set_params(**params)
        obj = self._train()
        return obj
        
    def _train(self):
        df = self.df.copy()
        lm = LinearRegression(fit_intercept=self.params["intercept"])
        df["z"] = df["z"] + numpy.random.rand(len(df)) * self.params["scale"] + self.params["dummy"]
        lm.fit(df[["x", "y"]], df["z"])
        obj = lm.score(df[["x", "y"]], df["z"])
        return obj

In [7]:
def opt_fit(space):
    @use_named_args(space)
    def my_objective(**params):
        obj = _objective(**params)
        return -obj
    gp = forest_minimize(func=my_objective, dimensions=space)
    return gp
    
def _objective(**params):
    print(params)
    df2 = df.copy()
    lm = LinearRegression(fit_intercept=params["intercept"])
    df2["z"] = df2["z"] + numpy.random.rand(len(df2)) * params["scale"] + params["dummy"]
    lm.fit(df2[["x", "y"]], df2["z"])
    obj = lm.score(df2[["x", "y"]], df2["z"])
    return obj

In [8]:
g = opt_fit(space)

{'intercept': 1, 'dummy': 0.52216771235836, 'scale': 3.1777870006066427}
{'intercept': 1, 'dummy': 0.2627247015347866, 'scale': 1.2976050667947359}
{'intercept': 0, 'dummy': 0.8538596243317582, 'scale': 4.167502734611342}
{'intercept': 1, 'dummy': 0.5997477142728695, 'scale': 1.6699008836700218}
{'intercept': 0, 'dummy': 0.6327441909160353, 'scale': 1.6872961871624832}
{'intercept': 1, 'dummy': 0.9159942736943919, 'scale': 1.9099355097357151}
{'intercept': 0, 'dummy': 0.877340717666809, 'scale': 2.6357712957477286}
{'intercept': 0, 'dummy': 0.09639391666231137, 'scale': 2.9574312568094143}
{'intercept': 1, 'dummy': 0.1166423711277699, 'scale': 3.055115191322134}
{'intercept': 0, 'dummy': 0.4249685082721997, 'scale': 2.545873279029911}
{'intercept': 1, 'dummy': 0.45745085448845313, 'scale': 1.5599790515127077}
{'intercept': 1, 'dummy': 0.1489007873390739, 'scale': 1.2219378905737557}
{'intercept': 1, 'dummy': 0.2763829636418537, 'scale': 1.2703593146057628}
{'intercept': 1, 'dummy': 0.2

In [ ]:
from skopt import forest_minimize

In [ ]:
ml = MLModel(df)

In [ ]:
ml.set_params(intercept=0,dummy=0,scale=1)
ml._train()

In [ ]:
ml.set_params(intercept=1,dummy=0,scale=1)
ml._train()

In [ ]:
g = ml.opt_fit(space)

In [ ]:
g.fun

In [ ]:
g.x

In [ ]:
g